# Slack Bot

## Load libraries

In [ ]:
import os
import pandas as pd
from slack import WebClient
from slack.errors import SlackApiError
pd.set_option('display.max_columns', None)

## Slack options

In [ ]:
slack_token = os.environ["SLACK_API_TOKEN"]
client = WebClient(token=slack_token)

## Load data 

In [ ]:
data = pd.read_json('https://healthdata.gov/resource/6xf2-c3ie.json')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Columns: 116 entries, state to total_patients_hospitalized_confirmed_influenza_coverage
dtypes: float64(30), int64(84), object(2)
memory usage: 49.1+ KB


## Clean data

In [ ]:
data = data[data.state != 'AS' ]
data = data[data.state != 'PR' ]
data = data[data.state != 'VI' ]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 0 to 53
Columns: 116 entries, state to total_patients_hospitalized_confirmed_influenza_coverage
dtypes: float64(30), int64(84), object(2)
memory usage: 46.6+ KB


## Values

### Covid beds

In [35]:
total_beds = data['inpatient_beds'].sum()
covid_occupied_beds = data['inpatient_beds_used_covid'].sum()
percent_covid_beds = round(((covid_occupied_beds/total_beds)*100),2)

### Staffing

In [56]:
critical_staffing_shortage_today_yes = data['critical_staffing_shortage_today_yes'].sum()
critical_staffing_shortage_today_no = data['critical_staffing_shortage_today_no'].sum()
critical_staffing_shortage_today_not_reported = data['critical_staffing_shortage_today_not_reported'].sum()
total_hospitals = critical_staffing_shortage_today_yes + critical_staffing_shortage_today_no + critical_staffing_shortage_today_not_reported

percent_hospitals_critical_shortages = round(((critical_staffing_shortage_today_yes/total_hospitals)*100),2)

4.91

## Write out message

In [57]:
msg = f"""📢 Daily COVID-19 hospitalization update 📢
🛏 Out of {(format (total_beds, ',d'))} available hospital beds in the country, {(format(covid_occupied_beds, ',d'))} beds are occupied by COVID-19 patients today, which account for {percent_covid_beds} percent of all beds.
🧑‍⚕️ {percent_hospitals_critical_shortages} percent hospitals in the country are reporting critical staffing shortages today.
"""

In [58]:
msg

'📢📢📢\n🛏 Out of 298 available hospital beds in the country, 3,199 beds are occupied by COVID-19 patients today, which account for 2.14 percent of all beds.\n🧑\u200d⚕️ 4.91 percent hospitals in the country are reporting critical staffing shortages today.\n'

## Send Slack message

In [59]:
slack_token = os.environ["SLACK_API_TOKEN"]
client = WebClient(token=slack_token)
try:
    response = client.chat_postMessage(
        channel="slack-bots",
        text=msg
    )
    print("success!")
except SlackApiError as e:
    assert e.response["ok"] is False
    assert e.response["error"]
    print(f"Got an error: {e.response['error']}")